This notebook enables to evaluate the SGBT model.

In [ ]:
import pickle
import csv

import glob2
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics

from utils.eval.eval_utils import compute_ROC

## Parameters

In [ ]:
ROOT_DIR = "/path/to/the/dataset"  # path where we expect to find directories named "postives", "negatives" and a csv file
OUTPUT_DIR = "SGBT20/dataset"  # directory where to output files, in the data folder
CHECKPOINT = ".../../../../data/model_saves/SGBT20/..."  # save of the SGBT model

## Load data

In [ ]:
with open(ROOT_DIR + "/dataset.csv", "r") as f:
    csv_reader = csv.reader(f, delimiter=",")
    lines = list(csv_reader)

pos_f = glob2.glob(f"{ROOT_DIR}/positives/*.npy")
pos_data = {p.split("/")[-1]:np.load(p) for p in pos_f}
neg_f = glob2.glob(f"{ROOT_DIR}/negatives/*.npy")
neg_data = {n.split("/")[-1]:np.load(n) for n in neg_f}

posX, negX = [], []
for line in lines:
    _X_list = posX if line[2]=="positive" else negX
    data = pos_data if line[2]=="positive" else neg_data
    station = line[0].split("/")[-1]
    idx = int(line[1])
    _X_list.append(data[station][idx])
posX, negX = np.array(posX), np.array(negX)
posY, negY = np.ones(len(posX)), np.zeros(len(negX))
X = np.concatenate((posX, negX))
Y = np.concatenate((posY, negY))
print(f"{len(posX)} positive samples and {len(negX)} negative samples found")

## Load the model and apply it

In [ ]:
with open(CHECKPOINT, 'rb') as f:
    model = pickle.load(f)
pred = model.predict_proba(X)[:,1]
print(metrics.roc_auc_score(Y, pred))

ROC curve

In [ ]:
TPr, FPr = compute_ROC(pred[Y==1], np.count_nonzero(Y==1), pred[Y==0], np.count_nonzero(Y==0), thresh_delta=0.001)
plt.plot(FPr, TPr)
np.save(f"../../../../data/npy/{OUTPUT_DIR}/FPr.npy", FPr)
np.save(f"../../../../data/npy/{OUTPUT_DIR}/TPr.npy", TPr)
plt.xlim(0,1)
plt.ylim(0,1)
plt.ylabel("TP rate")
plt.xlabel("FP rate")
plt.title("ROC curve")
plt.savefig(f"../../../../data/figures/{OUTPUT_DIR}/ROC.png")